# Project

In [4]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import uniform, randint, ttest_rel, ttest_ind

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = 10

import warnings
warnings.filterwarnings(action='ignore')

import matplotlib.pyplot as plt
%matplotlib inline

import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.express as px
import cufflinks as cf

# from pandas_profiling import ProfileReport

from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif 
from sklearn.feature_selection import SelectPercentile, VarianceThreshold, SelectFromModel, RFE

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold, LeaveOneOut
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

#preprocessing:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, normalize, scale, Normalizer, MinMaxScaler, FunctionTransformer

# models:
from sklearn.decomposition import PCA, KernelPCA
from sklearn.cluster import FeatureAgglomeration
from sklearn.ensemble import StackingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [5]:
cognitive_ability = 'VerbalWM_longest'
best_window = 10
datasets = {}

# loading the datasets
for task_number in range(1, 16):
    path = ("../Data/"+ str(cognitive_ability) + "/Task_" + str(task_number) + ".csv")
    df = pd.read_csv(path)
    datasets[task_number] = df

df = datasets[best_window]
# display(df)
# print(df.shape)

In [6]:
target = cognitive_ability

X = df.drop(columns=[target])
y = df[target]

# baseline score
DC = DummyClassifier(strategy="most_frequent")
DC.fit(X, y)
baseline = DC.score(X, y)
print(baseline)

0.5094339622641509


## Functions

In [7]:
#--------------------------------------------------------------------------------------

def results(name, y, y_pred):

    confusion_matatrix = confusion_matrix(y, y_pred)
    class_accuracy = confusion_matatrix.diagonal()/confusion_matatrix.sum(axis=1)
    overall_accuracy = confusion_matatrix.diagonal().sum()/confusion_matatrix.sum()

    data = {'Overall': overall_accuracy, 'Low': class_accuracy[0], 'High': class_accuracy[1]}
    results = pd.DataFrame(data, index = [name])

    return results
    
#--------------------------------------------------------------------------------------

def plot_results(df, title, xtitle='Window', ytitle='Accuracy', save=True):
    layout = cf.Layout(xaxis = cf.layout.XAxis(tickmode = 'linear'))
    
    fig = df.T.iplot(asFigure=True, xTitle=xtitle, yTitle=ytitle, 
              title=title, legend='top')
    fig.show()
    if (save):
        fig.write_html("images/" + title + ".html")
        fig.write_image("images/" + title + ".pdf")


## Classifiers/ hyper-parameter distributions

In [8]:
#--------------------------------------------------------------------------------------
# models:

classifiers = {
    'RF'  : RandomForestClassifier(n_jobs=-1),
    'LR'  : LogisticRegression(),
    'SVM' : LinearSVC(dual=False),
    'GB'  : GradientBoostingClassifier(),
    'XGB' : XGBClassifier(n_jobs=-1),
    'KNN' : KNeighborsClassifier(n_jobs=-1),
}

#--------------------------------------------------------------------------------------
# hyper-parameters

RF_dist = {
    'estimator__n_estimators'   : Integer(10, 500),
    'estimator__max_depth'      : Integer(1, 15)
}

LR_dist = {
    'estimator__C'       : Real(1e-6, 1e+6, prior='log-uniform'),
    #'estimator__penalty' : Categorical(['l1', 'l2'])
}


SVM_dist = {
    'estimator__C'       : Real(1e-6, 1e+6, prior='log-uniform')
}

GB_dist = {
    'estimator__loss'          : Categorical(['deviance', 'exponential']),
    'estimator__max_depth'     : Integer(1, 10),
    #'estimator__gamma'         : Real(0, 0.5),
    'estimator__learning_rate' : Real(0.05, 0.30)
}

XGB_dist = {
    'estimator__max_depth'     : Integer(1, 10), 
    'estimator__gamma'         : Real(0, 0.5),
    'estimator__learning_rate' : Real(0.05, 0.30)
}

KNN_dist = {
    'estimator__n_neighbors'   : Integer(1,10)
}

distributions = {
    'RF'  : RF_dist,
    'LR'  : LR_dist,
    'SVM' : SVM_dist,
    'GB'  : GB_dist,
    'XGB' : XGB_dist,
    'KNN' : KNN_dist,
}

#--------------------------------------------------------------------------------------

# Pipelines

In [9]:
#--------------------------------------------------------------------------------------
# pipeline

preprocessing = Pipeline([
    ('vart', VarianceThreshold(threshold=0.0)),
    ('scaler', StandardScaler()),
    ('minmax', MinMaxScaler())
])

feature_selection = Pipeline([
    ('sp', SelectPercentile(percentile=100)),
    ('kbest', SelectKBest(chi2, k=10)),
])

dim_reduction = Pipeline([
    ('pca', PCA(n_components=None)),
    ('kpca', KernelPCA(n_components=None, kernel='rbf'))
])

## Original models

In [10]:
n_rows = 53
rkf = RepeatedKFold(n_splits=n_rows, n_repeats=10)
loo = LeaveOneOut()

target = cognitive_ability
original_models = pd.DataFrame()

for name, estimator in classifiers.items():

    print(name)
    if name in ['RF', 'XGB', 'GB']:
        cv = rkf
    else:
        cv = loo
        
    for window in range(1, 16):
        df = datasets[window]
        X = df.drop(columns=[target])
        y = df[target]
        
        original = Pipeline([
        ('vart', VarianceThreshold(threshold=0.0)),
        ('estimator', estimator)])
    
        print(window, end="\r", flush=True)
        outer_loop = cross_validate(original, X, y, cv=cv, return_train_score=True, 
                                    n_jobs=-1, verbose=0, return_estimator=True)
        # print(outer_loop['test_score'].mean())
        
        original_models.loc[name, str(window)] = outer_loop['test_score'].mean()

original_models.loc['baseline', :] = baseline
display(original_models)

RF
LR
SVM
GB
XGB
KNN


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
RF,0.463208,0.502830,0.445283,0.458491,0.477358,0.558491,0.583019,0.635849,0.560377,0.613208,0.543396,0.605660,0.584906,0.584906,0.611321
LR,0.629630,0.629630,0.584906,0.528302,0.509434,0.566038,0.509434,0.547170,0.584906,0.547170,0.509434,0.490566,0.509434,0.509434,0.471698
SVM,0.537037,0.574074,0.622642,0.471698,0.452830,0.490566,0.452830,0.528302,0.547170,0.528302,0.547170,0.584906,0.566038,0.641509,0.584906
GB,0.574528,0.632075,0.403774,0.454717,0.409434,0.456604,0.786792,0.484906,0.520755,0.571698,0.575472,0.466038,0.505660,0.528302,0.590566
XGB,0.517925,0.667925,0.528302,0.547170,0.358491,0.471698,0.735849,0.660377,0.679245,0.716981,0.547170,0.566038,0.471698,0.603774,0.735849
KNN,0.518519,0.518519,0.528302,0.528302,0.528302,0.528302,0.528302,0.528302,0.528302,0.528302,0.528302,0.528302,0.528302,0.528302,0.528302
baseline,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434


In [12]:
# original_models.to_csv(r'log/VERWM_original_models.csv', index=True)
original_models = pd.read_csv('log/VERWM_original_models_1.csv', index_col=0)
plot_results(original_models, "Comparison of default models across tasks (VERWM)")

## Pre-Processing + Feature Selection

In [13]:
n_rows = 53
rkf = RepeatedKFold(n_splits=n_rows, n_repeats=10)
loo = LeaveOneOut()

target = cognitive_ability
preprocessed = pd.DataFrame()

for name, estimator in classifiers.items():
    
    print(name)
    if name in ['RF', 'XGB', 'GB']:
        cv = rkf
        #continue
    else:
        cv = loo
        
    for window in range(1, 16):
        df = datasets[window]
        X = df.drop(columns=[target])
        y = df[target]
        
        engineered = Pipeline([
        ('preprocessing', preprocessing),
        ('sp', SelectPercentile(percentile=40)),
        ('estimator', estimator)])
    
        print(window, end="\r", flush=True)
        outer_loop = cross_validate(engineered, X, y, cv=cv, return_train_score=True, 
                                    n_jobs=-1, verbose=0, return_estimator=True)
        
        preprocessed.loc[name, str(window)] = outer_loop['test_score'].mean()

preprocessed.loc['baseline', :] = baseline
display(preprocessed)

RF
LR
SVM
GB
XGB
KNN


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
RF,0.530189,0.496226,0.471698,0.462264,0.456604,0.571698,0.596226,0.647170,0.566038,0.535849,0.550943,0.581132,0.571698,0.537736,0.600000
LR,0.444444,0.388889,0.452830,0.490566,0.433962,0.547170,0.547170,0.509434,0.547170,0.509434,0.509434,0.452830,0.528302,0.509434,0.528302
SVM,0.462963,0.444444,0.433962,0.509434,0.471698,0.566038,0.584906,0.566038,0.528302,0.509434,0.433962,0.433962,0.471698,0.528302,0.509434
GB,0.637736,0.660377,0.369811,0.518868,0.403774,0.503774,0.600000,0.558491,0.539623,0.541509,0.483019,0.515094,0.509434,0.494340,0.458491
XGB,0.591509,0.703774,0.509434,0.603774,0.452830,0.528302,0.698113,0.679245,0.622642,0.698113,0.641509,0.660377,0.490566,0.452830,0.547170
KNN,0.574074,0.462963,0.584906,0.509434,0.490566,0.566038,0.433962,0.528302,0.471698,0.490566,0.509434,0.509434,0.528302,0.547170,0.490566
baseline,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434


In [14]:
# preprocessed.to_csv(r'log/VERWM_preprocessed.csv', index=True)
preprocessed = pd.read_csv('log/VERWM_preprocessed_1.csv', index_col=0)
plot_results(preprocessed, "Comparison of default models on pre-processed data across tasks (VERWM)")

### Class accuracies at best windows

In [15]:
loo = LeaveOneOut()

target = cognitive_ability
class_acc = pd.DataFrame()

for name, estimator in classifiers.items():

    print(name)
    
    if name in ['RF', 'XGB', 'GB']:
        n_rep = 10
    else:
        n_rep = 1
        
    engineered = Pipeline([
        ('preprocessing', preprocessing),
        ('sp', SelectPercentile(percentile=40)),
        ('estimator', estimator)])
        
    best_window = int(preprocessed.loc[name].idxmax())

    df = datasets[best_window]
    X = df.drop(columns=[target])
    y = df[target]

    L = []
    H = []
    O = []
    
    for rep in range(n_rep):
        y_pred  = cross_val_predict(engineered, X, y, cv=loo, n_jobs=-1, verbose=0)
        report = classification_report(y, y_pred, output_dict=True)
        
        L.append(report['0']['recall'])
        H.append(report['1']['recall'])
        O.append(report['accuracy'])
    
    class_acc.loc[name, 'window']  = best_window
    class_acc.loc[name, 'OVERALL'] = np.mean(O)
    class_acc.loc[name, 'LOW']     = np.mean(L)
    class_acc.loc[name, 'HIGH']    = np.mean(H)

display(class_acc)

RF
LR
SVM
GB
XGB
KNN


,window,OVERALL,LOW,HIGH
RF,8.0,0.609434,0.615385,0.603704
LR,6.0,0.547170,0.576923,0.518519
SVM,7.0,0.584906,0.576923,0.592593
GB,2.0,0.672222,0.688462,0.657143
XGB,2.0,0.703704,0.730769,0.678571
KNN,3.0,0.584906,0.423077,0.740741


In [16]:
# class_acc.to_csv(r'log/VERWM_class_acc.csv', index=True)
class_acc = pd.read_csv('log/VERWM_class_acc_1.csv', index_col=0)

## Hyperparameter optimization

In [17]:
loo = LeaveOneOut()

target = cognitive_ability
tuned = pd.DataFrame()

for name, estimator in classifiers.items():
    
    print(name)

    best_window = int(preprocessed.loc[name].idxmax())
    df = datasets[best_window]
    X = df.drop(columns=[target])
    y = df[target]

    engineered = Pipeline([
    ('preprocessing', preprocessing),
    ('sp', SelectPercentile(percentile=40)),
    ('estimator', estimator)])
    
    parameters = distributions[name]

    inner_loop = BayesSearchCV(engineered, parameters, n_iter=10, n_points=5, cv=10, refit=True, n_jobs=-1, verbose=0)
    outer_loop = cross_validate(inner_loop, X, y, cv=loo, return_train_score=True, 
                                n_jobs=-1, verbose=1, return_estimator=True)

    tuned.loc[name, 'window']  = best_window
    tuned.loc[name, 'default'] = preprocessed.loc[name].max()
    
    tuned.loc[name, 'tuned']   = outer_loop['test_score'].mean()
    
display(tuned)

RF


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed: 10.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


LR


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   22.8s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:   32.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


SVM


...

XGB


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:  1.9min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KNN


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed:   21.5s finished


,window,default,tuned
RF,8.0,0.647170,0.603774
LR,6.0,0.547170,0.452830
SVM,7.0,0.584906,0.566038
GB,2.0,0.660377,0.611111
XGB,2.0,0.703774,0.703704
KNN,3.0,0.584906,0.528302


In [20]:
# tuned.to_csv(r'log/VERWM_tuned.csv', index=True)
tuned = pd.read_csv('log/VERWM_tuned_1.csv', index_col=0)
plot_results(tuned.drop(columns=['window']), 
             title="default vs. tuned (VERWM)", 
             xtitle='configuration')

## Custom ensemble model

In [238]:
# ensemble model

estimators = [
    ('XGB', GradientBoostingClassifier()),
    ('KNN1', KNeighborsClassifier()),
    ('KNN2', KNeighborsClassifier(n_neighbors=10)),
    ('LR1', LogisticRegression(C=1)),
    ('LR2', LogisticRegression(C=0.1)),
    ('LR3', LogisticRegression(C=10)),
    ('SVM1', LinearSVC(dual=False)),
    ('SVM2', LinearSVC(dual=False, C=0.1)),
    ('SVM3', LinearSVC(dual=False, C=10))
]
              
ensemble = StackingClassifier(estimators=estimators, passthrough=True, n_jobs=-1,
                              final_estimator=LogisticRegression())


In [239]:
n_rows = 53
rkf = RepeatedKFold(n_splits=n_rows, n_repeats=10)
loo = LeaveOneOut()

target = cognitive_ability
    
ensembled = pd.DataFrame()
        
for window in range(1, 16):
    df = datasets[window]
    X = df.drop(columns=[target])
    y = df[target]

    ensembled_model = Pipeline([
    ('preprocessing', preprocessing),
    ('sp', SelectPercentile(percentile=40)),
    ('estimator', ensemble)])

    print(window, end="\r", flush=True)
    outer_loop = cross_validate(ensembled_model, X, y, cv=loo, return_train_score=True, 
                                n_jobs=-1, verbose=0, return_estimator=True)

    ensembled.loc['ensemble', str(window)] = outer_loop['test_score'].mean()

ensembled.loc['baseline', :] = baseline
display(ensembled)
plot_results(ensembled, "ensemble model across tasks")

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
ensemble,0.444444,0.555556,0.547170,0.660377,0.773585,0.660377,0.584906,0.622642,0.603774,0.603774,0.566038,0.622642,0.641509,0.660377,0.679245
baseline,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434,0.509434
